In [12]:
import os
from urllib import parse
import psycopg2
import numpy as np
import pandas as pd
import requests
import json
import datetime

In [13]:

parse.uses_netloc.append("postgres")
url = parse.urlparse(os.environ.get("DATABASE_URL"))

conn = psycopg2.connect(
    database=url.path[1:],
    user=url.username,
    password=url.password,
    host=url.hostname,
    port=url.port)


In [14]:
cursor = conn.cursor()

In [15]:
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall())

[('temperatures',), ('forecasts',), ('current',), ('ds_current',), ('ds_forecasts',)]


In [22]:
cursor=conn.cursor()
cursor.execute("CREATE TABLE temperatures( \
                resort TEXT, \
                time TIMESTAMP, \
                measurement REAL, \
                PRIMARY KEY(resort, time));")

In [160]:
conn.rollback()

In [90]:
conn.commit()

In [30]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
)

2018-02-17 14:11:09


In [99]:
base_str = "INSERT INTO temperatures (resort, time, measurement) VALUES ( \
            'Sutton',  '%s',  %1.2f)"

for i in range(10):
    timestamp = (datetime.datetime.now()+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    measurement = np.random.randn()

    cursor.execute(base_str%(timestamp, measurement))

    
conn.commit()

In [92]:
cursor=conn.cursor()
cursor.execute("SELECT * from temperatures;")


In [93]:
cursor.fetchone()

('jay', datetime.datetime(2018, 2, 17, 14, 28, 2), -1.12)

In [7]:
df = pd.read_sql('SELECT * from current;', conn)

In [8]:
df

,location,timestamp,rain_mm,snow_mm,temp_c_max,temp_c_min,windspeed_mph
0,Jay Peak,2018-02-17 21:10:00,0.0,0.0,-3.2,-3.7,7.0
1,Mt Sutton,2018-02-17 21:10:00,0.0,0.0,-3.2,-3.7,7.0
2,Mt Tremblant,2018-02-17 21:10:00,0.0,0.0,-6.0,-7.0,4.0
3,St Anne,2018-02-17 21:10:00,0.0,0.0,-10.6,-10.6,3.0
4,Le Massif,2018-02-17 21:10:00,0.0,0.0,-5.7,-5.7,6.0


In [100]:
baseurl = 'https://api.weatherunlocked.com/'

In [123]:
r=requests.get('http://api.weatherunlocked.com/api/current/51.50,-0.12?app_id=6765a93c&app_key=51b7b6cb5cf11b77b78d84d8f5c44845')

In [124]:
r.text

'{"lat":51.5,"lon":0.05,"alt_m":5.0,"alt_ft":16.4,"wx_desc":"Partly cloudy","wx_code":1,"wx_icon":"PartlyCloudyNight.gif","temp_c":6.0,"temp_f":42.8,"feelslike_c":4.59,"feelslike_f":40.26,"humid_pct":75.0,"windspd_mph":4.35,"windspd_kmh":7.0,"windspd_kts":3.78,"windspd_ms":1.94,"winddir_deg":220.0,"winddir_compass":"SW","cloudtotal_pct":50.0,"vis_km":10.0,"vis_mi":6.21,"vis_desc":null,"slp_mb":1023.0,"slp_in":30.29,"dewpoint_c":1.93,"dewpoint_f":35.47}'

In [2]:
loc_dict = {'Jay Peak':(44.9649,-72.4602), 
            'Mt Sutton':(45.1047, -72.5618),
            'Mt Tremblant': (46.1185,-74.5962),
            'St Anne': (47.0754,-70.9049),
            'Le Massif': (47.2848,-70.5697),
           }

In [110]:
r=requests.get('http://api.weatherunlocked.com/api/forecast/%1.2f,%1.2f?app_id=6765a93c&app_key=51b7b6cb5cf11b77b78d84d8f5c44845'%(loc_dict['Mt Sutton']))

In [111]:
result = json.loads(r.text)

In [132]:
result['Days'][0]['date']

'17/02/2018'

In [109]:
result

{'alt_ft': 498.69,
 'alt_m': 152.0,
 'cloudtotal_pct': 50.0,
 'dewpoint_c': -12.66,
 'dewpoint_f': 9.21,
 'feelslike_c': -5.15,
 'feelslike_f': 22.72,
 'humid_pct': 43.0,
 'lat': 45.05,
 'lon': -72.83,
 'slp_in': 30.18,
 'slp_mb': 1019.0,
 'temp_c': -1.8,
 'temp_f': 28.76,
 'vis_desc': None,
 'vis_km': 14.0,
 'vis_mi': 8.7,
 'winddir_compass': 'SSE',
 'winddir_deg': 160.0,
 'windspd_kmh': 9.0,
 'windspd_kts': 4.86,
 'windspd_mph': 5.59,
 'windspd_ms': 2.5,
 'wx_code': 1,
 'wx_desc': 'Partly cloudy',
 'wx_icon': 'PartlyCloudyNight.gif'}

In [3]:
def get_current_weather(loc_dict):
    
    data_dict = {}
    for location in loc_dict.keys():
        
        r = requests.get('http://api.weatherunlocked.com/api/forecast/%1.2f,%1.2f?app_id=6765a93c&app_key=51b7b6cb5cf11b77b78d84d8f5c44845'%(loc_dict[location]))
        
        result = json.loads(r.text)
        
        data_dict[location] = {'Windspeed_Mph':result['Days'][0]['windspd_max_mph'],
                               'Rain_mm': result['Days'][0]['rain_total_mm'],
                               'Snow_mm': result['Days'][0]['snow_total_mm'],
                               'Temp_C_Max': result['Days'][0]['temp_max_c'],
                               'Temp_C_Min': result['Days'][0]['temp_min_c'],
                               'Timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
                              } 
    
    return data_dict

def get_forecast_weather(loc_dict):
    
    data_dict = {}
    
    for location in loc_dict.keys():

        r = requests.get('http://api.weatherunlocked.com/api/forecast/%1.2f,%1.2f?app_id=6765a93c&app_key=51b7b6cb5cf11b77b78d84d8f5c44845'%(loc_dict[location]))
        
        result = json.loads(r.text)
        tomorrow = result['Days'][1]['Timeframes']
        
        data_dict[location] ={}
        for i in range(8):
            timestamp = (datetime.datetime.strptime(result['Days'][1]['date'],'%d/%m/%Y')
                         +datetime.timedelta(hours=int(tomorrow[i]['time'])/100)).strftime('%Y-%m-%d %H:%M')
            data_dict[location][timestamp]= {'Windspeed_Mph':tomorrow[i]['windspd_mph'],
                                                       'Rain_mm': tomorrow[i]['rain_mm'],
                                                       'Snow_mm': tomorrow[i]['snow_mm'],
                                                       'Temp_C_Max': tomorrow[i]['temp_c'],
                                                       'Temp_C_Min': tomorrow[i]['temp_c'],

                              } 
        
    
    return data_dict
    
def add_row(data, table_name):
    base_str = "INSERT INTO temperatures (resort, time, measurement) VALUES ( \
            'Sutton',  '%s',  %1.2f)"
    
    columns = data.keys()
    values = [data[column] for column in columns]

    insert_statement = 'INSERT into ' + table_name + ' (%s) values %s'

    cursor=conn.cursor()
    cursor.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))
    
    
def add_forecasts(forecast_dict):
    
    for location in forecast_dict.keys():

        for timestamp in forecast_dict[location]:
            data = forecast_dict[location][timestamp]
            data['Timestamp'] = timestamp
            data['Location'] = location

            add_row(data, 'forecasts')
            
def add_current(current_dict):
    
    for location in forecast_dict.keys():

        data = current_dict[location]
        data['Location'] = location

        add_row(data, 'current')

In [10]:
forecast_dict = get_forecast_weather(loc_dict)
forecast_dict

{'Jay Peak': {'2018-02-18 02:00': {'Rain_mm': 0.0,
   'Snow_mm': 0.0,
   'Temp_C_Max': -5.2,
   'Temp_C_Min': -5.2,
   'Windspeed_Mph': 6.0},
  '2018-02-18 05:00': {'Rain_mm': 0.0,
   'Snow_mm': 1.0,
   'Temp_C_Max': -5.7,
   'Temp_C_Min': -5.7,
   'Windspeed_Mph': 4.0},
  '2018-02-18 08:00': {'Rain_mm': 0.0,
   'Snow_mm': 0.0,
   'Temp_C_Max': -5.2,
   'Temp_C_Min': -5.2,
   'Windspeed_Mph': 7.0},
  '2018-02-18 11:00': {'Rain_mm': 0.0,
   'Snow_mm': 0.0,
   'Temp_C_Max': -3.8,
   'Temp_C_Min': -3.8,
   'Windspeed_Mph': 10.0},
  '2018-02-18 14:00': {'Rain_mm': 0.0,
   'Snow_mm': 0.0,
   'Temp_C_Max': -2.9,
   'Temp_C_Min': -2.9,
   'Windspeed_Mph': 12.0},
  '2018-02-18 17:00': {'Rain_mm': 0.0,
   'Snow_mm': 0.0,
   'Temp_C_Max': -3.2,
   'Temp_C_Min': -3.2,
   'Windspeed_Mph': 9.0},
  '2018-02-18 20:00': {'Rain_mm': 0.0,
   'Snow_mm': 0.0,
   'Temp_C_Max': -3.4,
   'Temp_C_Min': -3.4,
   'Windspeed_Mph': 4.0},
  '2018-02-18 23:00': {'Rain_mm': 0.0,
   'Snow_mm': 0.0,
   'Temp_C_Max': -

In [182]:
curr_dict = get_current_weather(loc_dict)

In [183]:
curr_dict

{'Jay Peak': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -3.2,
  'Temp_C_Min': -3.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 7.0},
 'Le Massif': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -5.7,
  'Temp_C_Min': -5.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 6.0},
 'Mt Sutton': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -3.2,
  'Temp_C_Min': -3.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 7.0},
 'Mt Tremblant': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -6.0,
  'Temp_C_Min': -7.0,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 4.0},
 'St Anne': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -10.6,
  'Temp_C_Min': -10.6,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 3.0}}

In [69]:
r=requests.get('https://api.openweathermap.org/data/2.5/weather?q=Hibbing,USA&APPID=4061abd8412a20f4e41b4b7e7429a79f')

In [70]:
json.loads(r.text)

{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 47.43, 'lon': -92.94},
 'dt': 1518961980,
 'id': 5030005,
 'main': {'humidity': 92,
  'pressure': 1013,
  'temp': 259.75,
  'temp_max': 261.15,
  'temp_min': 258.15},
 'name': 'Hibbing',
 'sys': {'country': 'US',
  'id': 1520,
  'message': 0.0038,
  'sunrise': 1518959450,
  'sunset': 1518997265,
  'type': 1},
 'visibility': 402,
 'weather': [{'description': 'snow', 'icon': '13d', 'id': 601, 'main': 'Snow'},
  {'description': 'fog', 'icon': '50d', 'id': 741, 'main': 'Fog'},
  {'description': 'mist', 'icon': '50d', 'id': 701, 'main': 'Mist'}],
 'wind': {'deg': 110, 'speed': 2.1}}

In [26]:
r = requests.get('https://api.openweathermap.org/data/2.5/weather?lat=45.5&lon=-73.61&APPID=4061abd8412a20f4e41b4b7e7429a79f')

In [27]:
r.text

'{"coord":{"lon":-73.61,"lat":45.5},"weather":[{"id":801,"main":"Clouds","description":"few clouds","icon":"02d"}],"base":"stations","main":{"temp":267.8,"pressure":1020,"humidity":73,"temp_min":267.15,"temp_max":268.15},"visibility":24140,"wind":{"speed":4.6,"deg":250},"clouds":{"all":20},"dt":1518958800,"sys":{"type":1,"id":3829,"message":0.0058,"country":"CA","sunrise":1518954629,"sunset":1518992806},"id":6077243,"name":"Montreal","cod":200}'

In [29]:
datetime.datetime.fromtimestamp(1518958800)

datetime.datetime(2018, 2, 18, 8, 0)

In [163]:
cursor=conn.cursor()
cursor.execute("CREATE TABLE forecasts( \
                Location TEXT, \
                Timestamp TIMESTAMP, \
                Rain_mm REAL, \
                Snow_mm REAL, \
                Temp_C_Max REAL, \
                Temp_C_Min REAL, \
                Windspeed_Mph REAL, \
                PRIMARY KEY(Location, Timestamp));")

conn.commit()

In [184]:
cursor=conn.cursor()
cursor.execute("CREATE TABLE current( \
                Location TEXT, \
                Timestamp TIMESTAMP, \
                Rain_mm REAL, \
                Snow_mm REAL, \
                Temp_C_Max REAL, \
                Temp_C_Min REAL, \
                Windspeed_Mph REAL, \
                PRIMARY KEY(Location, Timestamp));")

conn.commit()

In [169]:
conn.commit()

In [195]:
conn.rollback()


In [ ]:
curr_dict

In [191]:
curr_dict

{'Jay Peak': {'Location': 'Jay Peak',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -3.2,
  'Temp_C_Min': -3.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 7.0},
 'Le Massif': {'Location': 'Le Massif',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -5.7,
  'Temp_C_Min': -5.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 6.0},
 'Mt Sutton': {'Location': 'Mt Sutton',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -3.2,
  'Temp_C_Min': -3.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 7.0},
 'Mt Tremblant': {'Location': 'Mt Tremblant',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -6.0,
  'Temp_C_Min': -7.0,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 4.0},
 'St Anne': {'Location': 'St Anne',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -10.6,
  'Temp_C_Min': -10.6,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 3.0}}

In [196]:
add_current(curr_dict)

In [197]:
conn.commit()

In [8]:
r=requests.get('https://api.darksky.net/forecast/9eb353656151b0df7b72889bcc6f729a/44.9649,-72.4602')

In [9]:
r.text

'permission denied\n'

In [5]:
datetime.datetime.strptime('10/02/2018', '%d/%m/%Y').timestamp()

1518238800.0

In [6]:
r = requests.get('https://api.darksky.net/forecast/9eb353656151b0df7b72889bcc6f729a/44.9649,-72.4602,%d'%datetime.datetime.strptime('10/02/2018', '%d/%m/%Y').timestamp())

In [7]:
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
df = pd.DataFrame(r.json()['currently'], index=[0, 1])

In [38]:
df.append(r.json()['currently'], ignore_index=True)

,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,14.23,0.63,16.66,0.9,snow,345.05,0.0047,0.29,snow,1026.02,Flurries,19.18,1518238800,0,1.98,160,8.38,3.35
1,14.23,0.63,16.66,0.9,snow,345.05,0.0047,0.29,snow,1026.02,Flurries,19.18,1518238800,0,1.98,160,8.38,3.35
2,14.23,0.63,16.66,0.9,snow,345.05,0.0047,0.29,snow,1026.02,Flurries,19.18,1518238800,0,1.98,160,8.38,3.35


In [39]:
df.index

Int64Index([0, 1], dtype='int64')

In [43]:
df.loc[1, 'precipType'] = 'rain'

In [44]:
df

,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,14.23,0.63,16.66,0.9,snow,345.05,0.0047,0.29,snow,1026.02,Flurries,19.18,1518238800,0,1.98,160,8.38,3.35
1,14.23,0.63,16.66,0.9,snow,345.05,0.0047,0.29,rain,1026.02,Flurries,19.18,1518238800,0,1.98,160,8.38,3.35


[0, 1]

In [65]:
2*p_type

array([0, 2])

In [70]:
1- 2*p_type


array([ 1, -1])

In [71]:
p_type = np.array([int(i) for i in (df['precipType']=='rain').tolist()])
df['precip'] = df['precipIntensity'] * (1- 2*p_type)

In [72]:
df['precip']

0    0.0047
1   -0.0047
Name: precip, dtype: float64

In [83]:
r.json()['hourly']['data'][0]

{'apparentTemperature': 15.52,
 'cloudCover': 0.73,
 'dewPoint': 21.25,
 'humidity': 0.85,
 'icon': 'partly-cloudy-day',
 'ozone': 406.39,
 'precipAccumulation': 0.004,
 'precipIntensity': 0.0003,
 'precipProbability': 0.01,
 'precipType': 'snow',
 'pressure': 1020.67,
 'summary': 'Mostly Cloudy',
 'temperature': 25.07,
 'time': 1518962400,
 'uvIndex': 1,
 'visibility': 3.81,
 'windBearing': 338,
 'windGust': 14.29,
 'windSpeed': 9.37}

In [111]:
datetime.time()

datetime.time(0, 0)

In [128]:
df = pd.DataFrame(columns=['location', 'time', 'summary', 'icon', 'precipIntensity', 'precipProbability', 'precipType', 'temperature', 'apparentTemperature', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 'windBearing', 'cloudCover', 'uvIndex', 'visibility', 'ozone'])

In [138]:
now = datetime.datetime.date(datetime.datetime.now())
df = pd.DataFrame(columns=['location', 'time', 'summary', 'icon', 'precipIntensity', 'precipProbability', 'precipType', 'temperature', 'apparentTemperature', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 'windBearing', 'cloudCover', 'uvIndex', 'visibility', 'ozone'])
dates = [now - datetime.timedelta(days=i) for i in range(7)]

timestamps = [datetime.datetime.combine(d, datetime.time()).timestamp() for d in dates]

for loc in loc_dict.keys():
    lon = loc_dict[loc][0]
    lat = loc_dict[loc][1]
    for t in timestamps:
        r = requests.get('https://api.darksky.net/forecast/9eb353656151b0df7b72889bcc6f729a/%1.4f,%1.4f,%d'%(lon, lat, t),
                    params={'units':'si'})

        for h in r.json()['hourly']['data']:

            h['location'] = loc
            df = df.append(h, ignore_index=True)
            
return df



In [139]:
df

,location,time,summary,icon,precipIntensity,precipProbability,precipType,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone,precipAccumulation
0,Jay Peak,1518930000,Mostly Cloudy,partly-cloudy-night,0.0737,0.22,snow,-3.63,-6.82,-6.46,0.81,1015.93,2.09,4.92,183,0.88,0,3.40,358.28,0.086
1,Jay Peak,1518933600,Foggy,fog,0.0889,0.22,snow,-4.08,-6.84,-5.47,0.90,1014.86,1.75,4.31,180,0.98,0,2.70,366.75,0.109
2,Jay Peak,1518937200,Mostly Cloudy,partly-cloudy-night,0.1041,0.22,snow,-4.17,-4.17,-5.42,0.91,1014.42,1.34,3.21,182,0.83,0,3.57,375.78,0.132
3,Jay Peak,1518940800,Foggy,fog,0.1016,0.22,snow,-4.26,-4.26,-5.22,0.93,1014.27,0.53,2.28,200,0.82,0,2.48,383.17,0.127
4,Jay Peak,1518944400,Foggy,fog,0.1118,0.22,snow,-4.24,-4.24,-4.99,0.95,1014.77,0.41,1.74,242,0.88,0,1.85,389.58,0.140
5,Jay Peak,1518948000,Foggy,fog,0.1118,0.22,snow,-4.19,-4.19,-4.91,0.95,1015.62,0.70,1.77,294,0.98,0,2.99,395.11,0.137
6,Jay Peak,1518951600,Foggy,fog,0.1016,0.21,snow,-4.23,-4.23,-4.99,0.94,1016.49,0.73,2.79,281,0.94,0,2.75,399.61,0.127
7,Jay Peak,1518955200,Mostly Cloudy,partly-cloudy-day,0.0254,0.12,snow,-4.16,-4.16,-5.30,0.92,1017.80,1.13,2.83,293,0.75,0,9.83,402.71,0.033
8,Jay Peak,1518958800,Mostly Cloudy,partly-cloudy-day,0.0000,0.00,NaN,-3.91,-6.03,-5.82,0.87,1019.32,1.40,3.96,294,0.76,0,8.06,405.32,NaN
9,Jay Peak,1518962400,Mostly Cloudy,partly-cloudy-day,0.0000,0.00,NaN,-3.23,-5.71,-6.21,0.80,1020.73,1.66,4.43,283,0.74,1,7.81,406.39,NaN


In [126]:
r.json()['hourly']['data'][0].keys()

dict_keys(['time', 'summary', 'icon', 'precipIntensity', 'precipProbability', 'precipType', 'temperature', 'apparentTemperature', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 'windBearing', 'cloudCover', 'uvIndex', 'visibility', 'ozone'])

In [3]:
def get_past_week_darksky(loc_dict):
    """ Fetch hourly details for every day in the last week.
    """
    
    now = datetime.datetime.date(datetime.datetime.now())
    df = pd.DataFrame(columns=['location', 'time', 'summary', 'icon', 'precipIntensity', 'precipProbability', 'precipType', 'temperature', 'apparentTemperature', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 'windBearing', 'cloudCover', 'uvIndex', 'visibility', 'ozone'])
    dates = [now - datetime.timedelta(days=i) for i in range(7)]

    timestamps = [datetime.datetime.combine(d, datetime.time()).timestamp() for d in dates]

    for loc in loc_dict.keys():
        lon = loc_dict[loc][0]
        lat = loc_dict[loc][1]
        for t in timestamps:
            r = requests.get('https://api.darksky.net/forecast/9eb353656151b0df7b72889bcc6f729a/%1.4f,%1.4f,%d'%(lon, lat, t),
                        params={'units':'si'})

            for h in r.json()['hourly']['data']:

                h['location'] = loc
                df = df.append(h, ignore_index=True)

    return df



def get_nextweek_darksky(loc_dict):
    
    """ From the DarkSky docs:
    A Forecast Request returns the current weather conditions, 
    a minute-by-minute forecast for the next hour (where available), 
    an hour-by-hour forecast for the next 48 hours, and a day-by-day forecast for the next week.
    """
    
    
    df = pd.DataFrame(columns=['location', 'time', 'summary', 'icon', 'precipIntensity', 'precipProbability', 'precipType', 'temperature', 'apparentTemperature', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 'windBearing', 'cloudCover', 'uvIndex', 'visibility', 'ozone'])
   
    for loc in loc_dict.keys():
        lon = loc_dict[loc][0]
        lat = loc_dict[loc][1]

        r = requests.get('https://api.darksky.net/forecast/9eb353656151b0df7b72889bcc6f729a/%1.4f,%1.4f'%(lon, lat),
                    params={'units':'si'})

        dict_list = []
        
        c = r.json()['currently']
        
        c['location'] = loc
        
        df = df.append(c, ignore_index=True)
        
        for h in r.json()['hourly']['data']:

            h['location'] = loc
            df = df.append(h, ignore_index=True)
            
        for d in r.json()['daily']['data']:

            h['location'] = loc
            df = df.append(h, ignore_index=True)
            

    return df




In [140]:
r = requests.get('https://api.darksky.net/forecast/9eb353656151b0df7b72889bcc6f729a/%1.4f,%1.4f'%(lon, lat),
                        params={'units':'si'})

In [6]:
df = get_nextweek_darksky(loc_dict)

In [10]:
df['time'].apply(lambda x: datetime.datetime(x))

TypeError: Required argument 'month' (pos 2) not found

In [151]:

df[df['location']=='Jay Peak'].plot(x='time', y='precipIntensity')

/Users/archy/anaconda/envs/ski/lib/python3.6/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


KeyboardInterrupt: 

In [158]:
df[df['location']=='Jay Peak'].hist()

NameError: name '_converter' is not defined

In [2]:
import darksky
import db_utils

/Users/stephaniewillis/.virtualenvs/ski/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [11]:
curr_df = darksky.get_past_week_darksky(darksky.loc_dict)


In [16]:
forecast_df = darksky.get_nextweek_darksky(darksky.loc_dict)
db_utils.add_darksky(conn, forecast_df, 'ds_forecasts')

/Users/stephaniewillis/Documents/archy-code/ski-monitor/darksky.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['precipSigned'] = df['precipIntensity'] * (1- 2*p_type)


Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping value already in table
Skipping

In [3]:
conn = db_utils.connect_to_db()
cursor = conn.cursor()

In [6]:
cursor.execute('DROP TABLE ds_current')

In [7]:
cursor.execute('DROP TABLE ds_forecasts')

In [11]:
conn.commit()

In [11]:
db_utils.create_darksky_table(conn, 'ds_current')

NameError: name 'db_utils' is not defined

In [10]:
db_utils.create_darksky_table(conn, 'ds_forecasts')

In [23]:
conn.rollback()
cursor = conn.cursor()

In [18]:
import pandas as pd
df = pd.read_sql('SELECT * from ds_forecasts', conn)


Timestamp('2018-05-04 02:00:00')

In [32]:
td = datetime.datetime.now() - df['time'].max() 
td.seconds/(3600) + td.days*24

1553.098888888889

In [32]:
cursor.fetchall()

ProgrammingError: no results to fetch

In [ ]:

cursor.execute("DELETE FROM ds_current WHERE CURRENT_TIMESTAMP - time > '1 month'::interval")